<a href="https://colab.research.google.com/github/SophiePerrin/bac_a_sable/blob/main/compet_kaggle_LLM_resumes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Confection de résumés automatiques d'articles scientifiques par un modèle de langue pré-entraîné : étude et comparaison des performances de deux modèles.

**Auteur.e.s :** Sophie perrin, Emma, El Zube, Marius, Sylvano, pour le cours *Representation learning for NLP* @ Master 2 MALIA et MIASHS.

## Préparation de l'environnement

1. Se connecter ou se créer un compte sur https://huggingface.co/
2. Se créer un nouveau token d'accès : https://huggingface.co/settings/tokens
3. Enregistrer ce token comme un secret nommé `HF_TOKEN` dans Google Colab (icone "clef" dans le bandeau vertical)
4. Exécuter le code ci-dessous

Note importante pour la vitesse de calcul : pour activer l'accélération GPU dans votre notebook Colab :

    Ouvrez le menu « Exécution » : Dans votre notebook Colab, cliquez sur le menu « Exécution » en haut de l'écran.

    Sélectionnez « Modifier le type d'exécution » : Dans le menu déroulant, choisissez « Modifier le type d'exécution ».

    Choisissez le matériel GPU : Une fenêtre contextuelle s'affiche. Dans la section « Accélérateur matériel », sélectionnez « GPU » dans le menu déroulant.

    Enregistrez les modifications : Cliquez sur « Enregistrer » pour appliquer les modifications.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

from google.colab import userdata
from huggingface_hub import login; login(token=userdata.get("HF_TOKEN"))

from nltk import download; download('punkt_tab')
!pip install -U bitsandbytes txtai




[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


#Préparation des données d'entraînement pour leur utilisation par le modèle

On crée un dictionnaire qui apparie les articles et leurs résumés par leur identifiant commun.

In [ ]:
import os

# Chemins des dossiers contenant les fichiers
dossier_abstracts = "/content/drive/MyDrive/donnees/OBS/abstracts_OBS/"
dossier_articles = "/content/drive/MyDrive/donnees/OBS/articles_OBS/"

# Liste des fichiers dans chaque dossier
fichiers_abstracts = [f for f in os.listdir(dossier_abstracts) if f.startswith("abstract-")]
fichiers_articles = [f for f in os.listdir(dossier_articles) if f.startswith("article-")]

# Dictionnaires pour stocker les fichiers par identifiant
abstracts = {}
articles = {}

# Remplir les dictionnaires avec les fichiers en fonction des identifiants
for fichier in fichiers_abstracts:
    # Extraire l'identifiant du fichier abstract
    identifiant = fichier.split("-")[1].split(".")[0]
     #fichier.split("-") : La méthode split("-") divise le nom du fichier
    #en une liste de sous-chaînes en utilisant le caractère "-" comme séparateur.
    #Par exemple, si le fichier est "abstract-123.txt", fichier.split("-") renverra la liste ["abstract", "123.txt"].
    #fichier.split("-")[1] : En prenant l'élément d'indice 1 de cette liste (c'est-à-dire "123.txt"), on obtient la partie du nom du fichier après le préfixe "abstract-".
    #split(".")[0] : Ensuite, on divise cette chaîne "123.txt" avec split("."), ce qui donne ["123", "txt"].
    #On prend le premier élément de la liste (c'est-à-dire "123") qui est l'identifiant unique de l'abstract.
    abstracts[identifiant] = fichier
    #Cette ligne ajoute une entrée dans le dictionnaire abstracts, où la clé est : identifiant (par exemple "123")
    #et la valeur est le nom du fichier : fichier (par exemple "abstract-123.txt").

for fichier in fichiers_articles:
    # Extraire l'identifiant du fichier article
    identifiant = fichier.split("-")[1].split(".")[0]
    articles[identifiant] = fichier

# Dictionnaire pour stocker les appariements
appariements = {}

# Liste des abstracts et articles non-appariés
non_apparies_abstracts = []
non_apparies_articles = []

# Appariement des fichiers abstracts et articles par identifiant
for identifiant in abstracts:
    if identifiant in articles:
        # Ajouter l'appariement au dictionnaire
        appariements[identifiant] = {
            "abstract": abstracts[identifiant],
            "article": articles[identifiant]
        }
    else:
        # Ajouter à la liste des abstracts non appariés
        non_apparies_abstracts.append(abstracts[identifiant])

# Vérifier les articles non-appariés
for identifiant in articles:
    if identifiant not in abstracts:
        # Ajouter à la liste des articles non appariés
        non_apparies_articles.append(articles[identifiant])

# Affichage des appariements
"""
print("Appariements :")
for identifiant, fichiers in appariements.items():
    print(f"Identifiant {identifiant}:")
    print(f"  Abstract: {fichiers['abstract']}")
    print(f"  Article: {fichiers['article']}")
"""
# Affichage des abstracts non-appariés
print("\nAbstracts non-appariés :")
for abstract in non_apparies_abstracts:
    print(abstract)

# Affichage des articles non-appariés
print("\nArticles non-appariés :")
for article in non_apparies_articles:
    print(article)


Abstracts non-appariés :
abstract-37796016.txt
abstract-24279685.txt
abstract-36404343.txt
abstract-37833688.txt
abstract-37102598.txt
abstract-27423055.txt
abstract-35324507.txt
abstract-32946618.txt
abstract-32017677.txt
abstract-38022520.txt
abstract-32091358.txt
abstract-36951168.txt
abstract-36066965.txt
abstract-36891751.txt
abstract-37614109.txt
abstract-36314570.txt
abstract-37026745.txt
abstract-36519326.txt
abstract-36944050.txt
abstract-36944082.txt
abstract-37726286.txt
abstract-35081022.txt
abstract-36068298.txt
abstract-36372692.txt
abstract-31625835.txt
abstract-36525381.txt
abstract-36944043.txt

Articles non-appariés :


A noter que quelques résumés n'ont pas d'article qui leur correspond !

On retravaille un peu la forme de ce dictionnaire pour pouvoir le convertir au format "dataset" de Hugging face, nécessaire pour pouvoir entraîner le modèle choisi dessus.


In [ ]:
!pip install -U datasets #le "-U" demande l'installation de la toute dernière version du package
from datasets import Dataset #importe la classe Dataset de la bibliothèque datasets de Hugging Face.
#Cette classe est utilisée pour manipuler des ensembles de données dans un format qui peut être utilisé pour l'entraînement de modèles issus d'Hugging Face.

#Notre dictionnaire appariements n'est pas tout à fait sous la bonne forme pour être transformé en dataset de Hugging Face :
#on le remanie donc pour qu'il ait cette bonne forme.

# Initialisation des listes vides
identifiants = []
articles = []
abstracts = []

# Remplir les listes avec les données extraites du dictionnaire appariements
for identifiant, values in appariements.items():
    identifiants.append(identifiant)         # Ajout de l'identifiant
    articles.append(values["article"])      # Ajout de l'article
    abstracts.append(values["abstract"])    # Ajout de l'abstract

# Créer un dictionnaire avec les listes
data = {
    "identifiant": identifiants,
    "article": articles,
    "abstract": abstracts
}

# Créer le Dataset en convertissant notre nouveau dictionnaire via l'instruction Dataset.from_dict() de Dataset
dataset = Dataset.from_dict(data)


On convertit maintenant le texte des articles et des résumés en séquences de tokens.

In [ ]:
# Fonction pour préparer les données (tokenisation)
def tokenize_function(examples):
    # Tokeniser les articles et les résumés
    inputs = tokenizer(examples['article'], truncation=True, padding="max_length", max_length=1024) #données en entrée
    #examples['article'] : C'est la donnée brute (l'article) fournie à la fonction sous forme de texte.
    #Cette donnée vient d'un Dataset ou d'une DataLoader et est passée sous forme de liste de textes (articles).
    #tokenizer(examples['article']) : Cette ligne utilise le tokenizer pour convertir le texte brut des articles en une séquence de tokens.
    #Le tokenizer transforme le texte en un format compréhensible par le modèle (i.e., des IDs de tokens).

    #truncation=True : Si le texte est plus long que la longueur maximale définie, il sera tronqué pour correspondre à cette longueur maximale.

    #padding="max_length" : Cela permet de remplir (padd) le texte pour qu'il atteigne la longueur maximale spécifiée.
    #Si un article est plus court que la longueur maximale, des tokens de remplissage seront ajoutés.

    #max_length=1024 : La longueur maximale des séquences d'entrée est fixée à 1024 tokens. Si un article est plus long que cela, il sera tronqué à 1024 tokens.

    targets = tokenizer(examples['abstract'], truncation=True, padding="max_length", max_length=256)#données cible de l'entrainement du modèle

    # Retourner les inputs et targets sous la forme de dictionnaires
    inputs["labels"] = targets["input_ids"]
    #inputs["labels"] : Dans un modèle de génération de texte comme BigBirdPegasusForConditionalGeneration, les labels sont les séquences que le modèle doit prédire
    #(les résumés dans ce cas). En d'autres termes, le modèle apprend à prédire les tokens du résumé à partir des tokens de l'article.
    #targets["input_ids"] : input_ids est la représentation des tokens du résumé (les IDs numériques des tokens). Ces tokens sont utilisés comme labels lors de l'entraînement,
    #c'est-à-dire que le modèle essaiera de prédire ces input_ids lorsqu'il verra les tokens des articles.
    #Cette ligne ajoute donc les input_ids des résumés dans la clé "labels" des données d'entrée, qui est utilisée pendant l'entraînement pour calculer la perte
    #entre les prédictions du modèle et les résumés réels.
    return inputs

On paramètre la métrique "ROUGE", qui est un score permettant de mesurer la similarité entre les résumés générés et les résumés réels.

In [ ]:
!pip install -U evaluate
!pip install rouge_score
from evaluate import load # Import load_metric from evaluate instead of datasets

# Charger la métrique ROUGE
rouge_metric = load("rouge")

def compute_metrics(p):
    predictions, labels = p
    # Décoder les prédictions et les labels pour obtenir des chaînes de caractères
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Calculer la ROUGE
    result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels)

    # Retourner les résultats sous forme de dictionnaire
    return result

## Chargement du modèle bigbird-pegasus-large-arxiv



Ce modèle a pour singularité, par rapport à des modèles de type BERT, d'utiliser une version modifiée de l'architecture Transformer : il a un mécanisme d'attention sparse (attention réduite) par blocs. Concrètement, cela signifie qu'au lieu de calculer l'attention entre toutes les positions du texte, il divise la séquence (le texte) en blocs et applique l'attention uniquement au sein de ces blocs, ainsi qu'entre certains blocs (par exemple les blocs voisins ou un échantillonnage aléatoire de blocs).

Cela permet d'être économe en temps de calcul : avec un modèle de type BERT, ce dernier croissait en $O(N^2)$, où $N$ représente la taille du texte (ou document). Avec le mécanisme d'attention sparse par blocs, on tombe à des temps de calcul en $O(N.log(N))$ ou en $O(N)$.

Pour nos articles scientifiques, qui sont des documents longs, ce type de modèles, avec attention sparse par blocs, peut donc être particulièrement pertinent.

In [ ]:
!pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchaudio==0.13.1+cu117 -f https://download.pytorch.org/whl/torch_stable.html
!pip uninstall -y tensorflow
from transformers import BigBirdPegasusForConditionalGeneration, AutoTokenizer
import torch
from transformers import TrainingArguments, Trainer # Import TrainingArguments and Trainer

#Instancier le tokenizer pour le modèle pegasus
tokenizer = AutoTokenizer.from_pretrained("google/bigbird-pegasus-large-arxiv")

#chargement du modèle :
# by default encoder-attention is `block_sparse` with num_random_blocks=3, block_size=64/par défaut l'encoder-attention est "block sparse"
#avec num_random_blocks=3 et block_size=64
model = BigBirdPegasusForConditionalGeneration.from_pretrained("google/bigbird-pegasus-large-arxiv",
                                                               return_dict=True,
                                                               torch_dtype=torch.float16,
                                                               device_map="auto")

######################Options pour modifier la taille et le nombre des blocs d'attention :
# decoder attention type can't be changed & will be "original_full"
# you can change `attention_type` (encoder only) to full attention like this:
##model = BigBirdPegasusForConditionalGeneration.from_pretrained("google/bigbird-pegasus-large-arxiv", attention_type="original_full")

# you can change `block_size` & `num_random_blocks` like this:
##model = BigBirdPegasusForConditionalGeneration.from_pretrained("google/bigbird-pegasus-large-arxiv", block_size=16, num_random_blocks=2)
######################



Looking in links: https://download.pytorch.org/whl/torch_stable.html


On paramètre l'entraînement du modèle.

In [ ]:
# Appliquer la tokenisation
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Mélanger et diviser le dataset tokenisé, en 90% pour l'entraînement et 10% pour la validation
train_dataset, val_dataset = tokenized_datasets.train_test_split(test_size=0.1).values()

# Définir les arguments d'entraînement
training_args = TrainingArguments(
    output_dir="./bigbird_pegasus_finetuned",  # Répertoire de sortie pour enregistrer les résultats
    evaluation_strategy="epoch",  # Évaluer après chaque époque
    learning_rate=2e-5,  # Taux d'apprentissage
    per_device_train_batch_size=2,  # Taille du batch pour l'entraînement
    per_device_eval_batch_size=2,   # Taille du batch pour l'évaluation
    num_train_epochs=3,  # Nombre d'époques d'entraînement
    weight_decay=0.01,  # L2 regularization
    save_total_limit=2,  # Limite du nombre de sauvegardes du modèle
    logging_dir="./logs",  # Répertoire pour les logs
    logging_steps=10,  # Fréquence des logs
    report_to="tensorboard",  # Optionnel : Utiliser TensorBoard pour la visualisation des métriques
)

# Créer un objet Trainer
trainer = Trainer(
    model=model,  # Le modèle à fine-tuner
    args=training_args,  # Les arguments d'entraînement
    train_dataset=train_dataset,  # Jeu d'entraînement
    eval_dataset=val_dataset,  # Jeu de validation
   # compute_metrics=compute_metrics,  # Fonction pour calculer les métriques - elle nous permettra d'utiliser le score "ROUGE"
)

# Lancer l'entraînement
trainer.train()


# Sauvegarder le modèle fine-tuné
trainer.save_model("./bigbird_pegasus_finetuned")

Map:   0%|          | 0/402 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


RuntimeError: "bernoulli_scalar_cpu_" not implemented for 'Half'

In [ ]:
# Évaluer la performance du modèle sur l'ensemble de validation
eval_results = trainer.evaluate(eval_dataset=val_dataset)

# Afficher les résultats d'évaluation
print(eval_results)


#Chargement du modèle Bart-large-CNN

BART est un modèle de transformer encodeur-décodeur (seq2seq : "sequence to séquence") avec un encodeur bidirectionnel (similaire à BERT) et un décodeur autoregressif (similaire à GPT).
Il est pré-entraîné en masquant des parties du texte et en apprenant à prédire ces parties - comme si on lui demandait de débruiter un texte bruité.
Il utilise une attention dense classique pour un transformer, où chaque token de l'entrée prend en compte tous les autres tokens de la séquence.

In [ ]:
#A faire

In [ ]:
!nvidia-smi #afficher des informations sur les GPU NVIDIA disponibles sur la machine

/bin/bash: line 1: nvidia-smi: command not found
